In [ ]:
"""
author: 陈凯鑫
datetime: 2023-8-6
假期应该多陪陪家人,而不是为了一个不确定的机会去挣扎
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 设置图片dpi
plt.rcParams['figure.dpi'] = 300

In [ ]:
"""
读取excel文件 附件2 慢性病及相关因素流调数据.xlsx
"""
df_food_raw = pd.read_excel('附件2 慢性病及相关因素流调数据.xlsx',sheet_name='饮食习惯')
# 不读取后面7列
df_food = df_food_raw.iloc[:,:-7]
print(df_food.head())

In [ ]:
"""
进行数据缺失值填补或者纠错
如果使存在使用大米的频率不为缺失值,那么吃大米就为1,否则为2
以5列作为一个循环
"""
df_food1 = df_food.fillna(-1).values.copy()
j = 0
for i in range(df_food1.shape[0]):
    for j in range(0,df_food1.shape[1],5):
        if df_food1[i,j]>0:
            pass
        elif np.any(df_food1[i,j+1:j+5] >0):
            df_food1[i,j] = 1
        else:
            df_food1[i,j] = 2
df_food1_df = pd.DataFrame(df_food1,columns=df_food.columns)
print(df_food1_df.head())

In [ ]:
"""
如果没有频率,也没有每次的量,那么就改为2
"""
j = 0
for i in range(df_food1.shape[0]):
    for j in range(0,df_food1.shape[1],5):
        if np.all(df_food1[i,j+1:j+5]<0):
            df_food1[i,j] =2

In [ ]:
"""
因为频率包括每天,每周,每月,于是我们统一为一个月的频数
"""
j = 0
for i in range(df_food1.shape[0]):
    for j in range(0,df_food1.shape[1],5):
        if df_food1[i,j]==2: # 如果不吃
            df_food1[i,j+3] = 0
            df_food1[i,j+4] = 0
        else:
            if df_food1[i,j+3]>0:
                pass
            elif df_food1[i,j+2]>0:
                # 月频率为周的4倍
                df_food1[i,j+3] = 4*df_food1[i,j+2].copy()
            elif df_food1[i,j+1]>0:
                # 月频率为日的30倍
                df_food1[i,j+3] = 30*df_food1[i,j+1].copy()

In [ ]:
"""
因为使用频率(月)存在缺失值,那么就用平均值去填补
"""
for j in range(0,df_food1.shape[1],5):
    m = np.mean(df_food1[df_food1[:,j+3]>0,j+3])
    df_food1[df_food1[:,j+3]<0,j+3] = round(m,2)


In [ ]:
"""
因为每次的量也可能存在缺失值,那么就用平均值去填补
"""
for i in range(df_food1.shape[0]):
    for j in range(0,df_food1.shape[1],5):
        if df_food1[i,j+4]<0:
            df_food1[i,j+4] = round(np.mean(df_food1[df_food1[:,j+3]==df_food1[i,j+3],j+4]),2)
        if df_food1[i,j+4]<0:
            df_food1[i,j+4] = round(np.mean(df_food1[:,j+4]),2)

In [ ]:
"""
去掉日,周这两列
"""
df_food2 = df_food1[:,np.array([0,3,4])+0*5].copy()
for j in range(5,df_food1.shape[1],5):
    df_food2 = np.c_["1",df_food2,df_food1[:,np.array([0,3,4])+j].copy()]

df_food2_df = pd.DataFrame(df_food2,columns=[i for i in df_food.columns.values if i[0]!="U"])
df_food2_df


注意:
df_food 是最开始的dataframe
df_food1 是填补缺失值后的数组
df_food1_df 是填补缺失值后的dataframe
df_food2 是去掉日,周后的数组
df_food2_df 是去掉日,周后的dataframe

In [ ]:
"""
统计每种食物使用的人数占总人数的比
"""
people_eat = np.array([np.sum(df_food2[:,j]==1) for j in range(0,df_food2.shape[1],3)])/df_food2.shape[0]
print(df_food2_df.columns[::3].values)
print(people_eat)
"""
用seaborn将people_eat画出来,其中它的横坐标标签为df_food2_df.columns[::3].values
"""
plt.figure(figsize=(10,5))
sns.barplot(x=df_food2_df.columns[::3].values,y=people_eat)
plt.xticks(rotation=90)
plt.xlabel('食物种类')
plt.ylabel('食用人数占比')
#plt.title('每种食物的食用人数占比')
plt.show()

In [ ]:
"""
考虑到每种食物的使用频率也在健康管理范围内,因此我们对此进行统计并进行可视化
"""
people_eat_frequency = np.array([np.sum(df_food2[:,j+1]) for j in range(0,df_food2.shape[1],3)])/df_food2.shape[0]
# 用seaborn绘制柱状图,其中横坐标是df_food2_df.columns[::3].values,纵坐标是people_eat_frequency
plt.figure(figsize=(10,5))
sns.barplot(x=df_food2_df.columns[::3].values,y=people_eat_frequency)
plt.xticks(rotation=90)
plt.xlabel('食物种类')
plt.ylabel('每月平均食用频率')
#plt.title('每种食物的每月食用频率')
plt.show()

In [ ]:
"""
考虑到每种食物的使用量也在健康管理范围内,因此我们对此进行统计并进行可视化
"""

plt.figure(figsize = (10,5))
# 横坐标为df_food2_df.columns[::3].values
bx = plt.boxplot(df_food2[:,[j+2 for j in range(0,df_food2.shape[1],3)]],showfliers = 0,labels=df_food2_df.columns[::3].values,patch_artist=True)
plt.xticks(rotation=90)
plt.xlabel("食物种类",fontsize = 14)
plt.ylabel("每次食用量(两)",fontsize = 14)
plt.show()

In [ ]:
"""
因为之前在导入excel中的饮食健康那一栏中,没有导入后面的7列
我们这次进行导入并进行统计分析和可视化
"""
# 选取df_food的最后七列
df_food_7 = df_food_raw.iloc[:,-7:].copy()
plt.figure(figsize = (10,5))
bx = plt.boxplot(df_food_7.fillna(0).values,showfliers = 0,patch_artist=True)
plt.xticks(np.arange(1,8),df_food_7.columns)
plt.ylabel("平均每月食用量(两)")
plt.show()